In [ ]:
import pandas as pd 
import numpy as np
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()


In [ ]:
ppc = pd.read_csv("../final_data/1718ppc_cleaned.csv")
upc = pd.read_csv("../final_data/pd_pos_all1718_public.csv") #upc

In [ ]:
upc=upc.merge(ppc,how="inner",on="upc")
upc = upc.drop_duplicates('upc')
upc.index = range(upc.shape[0])

In [ ]:
ppc2 = pd.read_csv("../final_data/1516ppc_cleaned.csv")
upc2 = pd.read_csv("../final_data/pd_pos_all1516.csv") #upc
upc2=upc2.merge(ppc2,how="inner",on="upc")
upc2 = upc2.drop_duplicates('upc')
upc2.index = range(upc2.shape[0])

In [ ]:
upc=upc.iloc[:,1:]

In [ ]:
upc = pd.concat([upc,upc2])

In [ ]:
def get_sub(t,u2):
    result =pd.DataFrame(columns = [t]+[str(i) for i in u2.ec.unique().tolist()],index = range(len(u2[t].unique())))
    result[t] = u2[t].unique()
    for i in range(len(u2[t].unique())):
        a = u2[u2[t] == u2[t].unique()[i]]
        b = a.ec.unique()
        for j in b:
            result.iloc[i,result.columns.get_loc(str(j))]= 1
    result = result.fillna(0)
    return result
category_data = get_sub('category',upc)
deptid_data =  get_sub('deptid',upc)
aisle_data = get_sub("aisle",upc)

In [ ]:
category_data=category_data.drop(170)
category_data=category_data.reset_index(drop=True)
# If category is missing, after checking data, if upcdesc is not NA, we can directory set ec as -80.

In [ ]:
category_data.to_csv("../final_data/all_category_ec.csv",index = False)
deptid_data.to_csv("../final_data/all_deptid_ec.csv",index = False)
aisle_data.to_csv("../final_data/all_aisle_ec.csv",index = False)

In [ ]:
category_data = get_sub('category',upc2)
deptid_data =  get_sub('deptid',upc2)
aisle_data = get_sub("aisle",upc2)
category_data=category_data.drop(category_data[category_data["category"]==0].index[0])
category_data=category_data.reset_index(drop=True)
# If category is missing, after checking data, if upcdesc is not NA, we can directory set ec as -80.


In [ ]:
category_data.to_csv("../final_data/1516_category_ec.csv",index = False)
deptid_data.to_csv("../final_data/1516_deptid_ec.csv",index = False)
aisle_data.to_csv("../final_data/1516_aisle_ec.csv",index = False)

In [ ]:
def get_rate(data):
    aa = pd.DataFrame(data[data["ec"]==-99.0].groupby("category").count().iloc[:,1])
    bb = pd.DataFrame(data[data["category"].isin(aa.index)].groupby("category").count().iloc[:,1])
    cc = pd.concat([aa,bb],axis=1)
    cc.columns = ["count_-99","total"]
    cc["-99_rate"]= cc["count_-99"]/cc["total"]
    cc = cc.sort_values(by=["-99_rate"],ascending=False)
    return cc

In [ ]:
cc = get_rate(upc2)

In [ ]:
xx = get_rate(upc)

In [ ]:
xx=xx.drop("VARIETY")

In [ ]:
xx=xx.reset_index()
cc=cc.reset_index()

In [ ]:
cc.to_csv("../final_data/1516_-99_rate.csv",index=False)
xx.to_csv("../final_data/all_-99_rate.csv",index=False)